# Importing Necessary Libraries

In [ ]:
import numpy as np 
import pandas as pd # library for data analsysis
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
from bs4 import BeautifulSoup #for scrapping data from the web page
print('Libraries imported.')

# Scrapping Data from Web

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
webpage = requests.get(url)

In [ ]:
soup = BeautifulSoup(webpage.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs: #loop to scrap the table column wise
    i = tr.find_all('td')
    if i:
        rows.append(i)

lst = []
for row in rows: #loop to separate data into cells from rows
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

In [ ]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)#creating the dataframe
#merging the Neighborhoods in the same postal area
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

df.shape

# Getting Coordinate Data from the Web and merge it into the above dataframe

In [ ]:
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/ #downloading the Coordinate data file


In [ ]:
df_cord = pd.read_csv('GeoCord.csv') #reading the contents of the downloaded file into a data frame

df['Latitude'] = np.nan
df['Longitude'] = np.nan

# For each postcode in df_assigned, find corresponding coordinates in df_cord and assign it to df_assigned
for index in df.index:
    cord_index = df_cord['Postal Code'] == df.loc[index, 'PostalCode']
    df.at[index, 'Latitude'] = df_cord.loc[cord_index, 'Latitude'].values
    df.at[index, 'Longitude'] = df_cord.loc[cord_index, 'Longitude'].values

# Display the results
df.head(20)